# H share close auction session

In [1]:
import pandas as pd

# hx sec https://hk.vbkr.com/help/topic2-what-is-the-reference-equilibrium-price-of-hk-stocks
# hkex https://www.hkex.com.hk/global/exchange/faq/securities-market/trading/pre_opening-session?sc_lang=zh-hk#collapse-9

In [45]:
def cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell):
    px_all = list(set(lmt_buy.keys()) | set(lmt_sell.keys()))
    if lmt_sell:
        low = min(lmt_sell.keys())
    else:
        low = float('inf')
    if lmt_buy:
        high = max(lmt_buy.keys())
    else:
        high = float('-inf')
    if low > high:  # if no buy>=sell, then take reference
        print(f"Can't determine IEP from limit order, we use reference price {reference} as IEP\n")
        px = [reference]
        px_all.append(reference)
    else:
        px = []
        for p in px_all:
            if low<=p<=high:
                px.append(p)
    px_all.sort()

    acc_buy = auc_buy
    acc_sell = auc_sell
    book = {p: [0, 0, 0] for p in reversed(px_all)}

    for p in px_all:
        if p in lmt_sell:
            acc_sell += lmt_sell[p]
        book[p][1] = acc_sell
    for p in reversed(px_all):
        if p in lmt_buy:
            acc_buy += lmt_buy[p]
        book[p][0] = acc_buy
    for p in px_all:
        b, s, _ = book[p]
        book[p][2] = min(b, s)

    iev = 0
    iep = -1
    for p in px:
        if book[p][2]>iev:
            iev = book[p][2]
            iep = p
    if iep==-1:
        print("No match")
    else:
        print(f"IEP will be {iep}, and IEV will be {iev}")
    book = pd.DataFrame.from_dict(book, orient="index")
    book.columns = ["buy_acc", "sell_acc", "match"]
    return book

In [52]:
# empty case
reference = 10
lmt_buy = {}
auc_buy = 0
lmt_sell = {}
auc_sell = 0

cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell)

Can't determine IEP from limit order, we use reference price 10 as IEP

No match


,buy_acc,sell_acc,match
10,0,0,0


In [46]:
# case 1
# https://hk.vbkr.com/help/topic2-what-is-the-reference-equilibrium-price-of-hk-stocks
reference = 10
lmt_buy = {8: 1000, 9: 200, 10: 500, 11: 700, 12: 100}
auc_buy = 0
lmt_sell = {8: 400, 9: 200, 10: 1200, 11: 600, 12: 300}
auc_sell = 0

cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell)

IEP will be 10, and IEV will be 1300


,buy_acc,sell_acc,match
12,100,2700,100
11,800,2400,800
10,1300,1800,1300
9,1500,600,600
8,2500,400,400


In [47]:
# case 2
# https://www.hkex.com.hk/-/media/HKEX-Market/Services/Trading/Securities/Overview/Trading-Mechanism/Trading-Mechanism-of-CAS-in-the-Securities-Market-(Chi).pdf
reference = 100
lmt_buy = {101:100}
auc_buy = 100
lmt_sell = {102:100}
auc_sell = 0

cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell)

Can't determine IEP from limit order, we use reference price 100 as IEP

No match


,buy_acc,sell_acc,match
102,100,100,100
101,200,0,0
100,200,0,0


In [48]:
# case 3
# https://www.hkex.com.hk/-/media/HKEX-Market/Services/Trading/Securities/Overview/Trading-Mechanism/Trading-Mechanism-of-CAS-in-the-Securities-Market-(Chi).pdf
reference = 100
lmt_buy = {}
auc_buy = 100
lmt_sell = {99:100}
auc_sell = 0

cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell)

Can't determine IEP from limit order, we use reference price 100 as IEP

IEP will be 100, and IEV will be 100


,buy_acc,sell_acc,match
100,100,100,100
99,100,100,100


In [49]:
# case 4
# https://www.hkex.com.hk/global/exchange/faq/securities-market/trading/pre_opening-session?sc_lang=zh-HK#collapse-9
reference = 32
lmt_buy = {31.9:13000,32:9000}
auc_buy = 2000
lmt_sell = {31.9:10000, 32:16000, 32.1:8000, 32.2:7000}
auc_sell = 0

cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell)

IEP will be 32, and IEV will be 11000


,buy_acc,sell_acc,match
32.2,2000,41000,2000
32.1,2000,34000,2000
32.0,11000,26000,11000
31.9,24000,10000,10000


In [50]:
# case 5
# https://www.hkex.com.hk/global/exchange/faq/securities-market/trading/pre_opening-session?sc_lang=zh-HK#collapse-9
reference = 64
lmt_buy = {63.75:2000, 64:5000, 64.25:1000}
auc_buy = 10000
lmt_sell = {63.75:2000, 64:3000, 64.25:4000}
auc_sell = 5000

cas(reference, lmt_buy, auc_buy, lmt_sell, auc_sell)

IEP will be 64.25, and IEV will be 11000


,buy_acc,sell_acc,match
64.25,11000,14000,11000
64.00,16000,10000,10000
63.75,18000,7000,7000
